In [8]:
import pyodbc
import pandas as pd
import datetime
import ast
import re


In [9]:
regex = r'(?<=^|(?<=[^a-zA-Z0-9-_\./]))@([A-Za-z]+[A-Za-z0-9-_.]+[A-Za-z0-9-_])'

In [10]:
with open('pw.txt') as f:
    for l in f:
        uname, pw = l.split()

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=vip.hivesql.io;"
            "Database=DBHive;"
            f"UID={uname};"
            f"PWD={pw};")
cnxn = pyodbc.connect(cnxn_str)

blacklist=('sunlit7')

In [11]:
fromdate = (datetime.date.today()-datetime.timedelta(days=31)).strftime('%Y-%m-%d')
#print(fromdate)
nowacc = pd.read_sql(f"""select a.name, floor(a.vesting_shares) as vests,
(select sum(pending_payout_value) as rewards
from Comments c
where c.author=a.name
and datediff(minute, C.created, GETUTCDATE()) < 60*24*7) as pending
, a.posting_json_metadata,
a.balance + a.savings_balance as hive, a.hbd_balance + a.savings_hbd_balance as hbd
from accounts a
where a.vesting_shares between 900000 and 950000
and a.vesting_withdraw_rate = 0
--and a.last_vote_time > '{{fromdat}}'
and exists (select 1 from Comments c2 where c2.author=a.name
and c2.depth=0 and datediff(minute, C2.created, GETUTCDATE()) < 60*24*6)
order by a.vesting_shares desc
""", cnxn)

In [12]:
query=f"""select c.title , c.created ,c.body 
from
    Tags
    INNER JOIN Comments c ON Tags.comment_id = C.ID
WHERE
    Tags.tag = 'minnowmaker'
    and c.author = 'steevc'
    and  c.[depth] =0
    and datediff(minute, C.created, GETUTCDATE()) < 60*24*10
order by c.created desc"""

then = pd.read_sql(query, cnxn)
users = []
for i, l in then.iterrows():
    for n in re.finditer(regex, l['body']):
        u = n.group().lower()
        if u not in ('@tenkminnows', '@arcange', '@hivesql', '@taskmaster4450'):
            users.append(n.group().lower())
            # Only fetch those in the table
            if len(users) == 20:
                break
    break
#print(users)

In [13]:
n = 0
prev = []
print('|Name|HP%|HIVE|HBD|New|')
print('|-|-|-|-|-|')
for i, l in nowacc.iterrows():
    if l['pending'] > 1 and l['name'] not in blacklist:
        #print(l)
        #meta = ast.literal_eval(l['posting_json_metadata'])
        #prof = meta['profile']
        #if 'name' in prof:
        #    print(prof['name'])
        acc, vests, hive, hbd = l['name'], int(l['vests'])/1E4, l['hive'], l['hbd']
        #print(f'{acc}')
        if f'@{acc}' not in users:
            newb = '*'
        else:
            newb = ''
            prev.append(f'@{acc}')
        print(f'|@{acc}|{vests}%|{hive}|{hbd}|{newb}|')
        n += 1
        if n >= 20:
              break

|Name|HP%|HIVE|HBD|New|
|-|-|-|-|-|
|@opick|94.925%|0.0|86.222|*|
|@pauliinasoilu|94.7796%|0.198|0.0||
|@annishajennat|94.6254%|0.0|170.541|*|
|@timoremoti|94.592%|15.227|0.0|*|
|@jayparagat|94.5741%|0.154|0.0||
|@alina97|94.234%|1.581|15.71||
|@debbie-ese|94.0564%|0.112|1.998||
|@joalgusa|93.9079%|0.0|11.765|*|
|@yoginiofoz|93.6608%|8.3|52.183|*|
|@ashleyjaramillo|93.6493%|27.012|0.526|*|
|@irissol|93.4552%|0.001|0.0|*|
|@holgerfinn|93.4036%|3.04|131.393||
|@cflclosers|93.2609%|1588.091|432.087|*|
|@kavii|93.2392%|7.941|196.107|*|
|@cruis|93.2149%|6.796|99.696|*|
|@littlebee4|93.1608%|18.172|91.76|*|
|@dreemit|92.9651%|620.551|430.672||
|@keniel16|92.827%|0.428|2905.144|*|
|@aldara|92.6857%|42.488|0.223|*|
|@sandracabrera|92.5458%|0.002|12.38||


In [14]:
grad=[]
for g in users:
    if g not in prev:
        #print(g)
        gv = pd.read_sql(f"""select floor(a.vesting_shares) as vests
    from accounts a
    where a.name = '{g[1:]}'
    """, cnxn)
        for i, l in gv.iterrows():
            if int(l['vests']) > 1E6:
                print(g, l['vests'])
                grad.append(g)
print(', '.join(grad))

@benwickenton 1406592.0
@craftingwithreny 1024619.0
@benwickenton, @craftingwithreny
